In [0]:
%sql
show tables from demo;
-- select * from demo.titanic_raw;
-- select * from demo.his_tvr;

database,tableName,isTemporary
demo,boston,false
demo,cancer,false
demo,his_tvr,false
demo,titanic_raw,false


### AutoML

In [0]:
# MLFlow functions
import os
import glob
import json
import yaml
import mlflow
import pandas as pd
import mlflow.sklearn


def get_latest_file(path):
  list_of_files = glob.glob(path + "*") # * means all if need specific format then *.csv
  latest_file = max(list_of_files, key=os.path.getctime)
  return latest_file

def init_mlflow(expt_name, artifact_location, tracking_uri=None):
    """
    Function to initialize the MLFlow Client for the experiment
    Parameters:
        expt_name (string): name of the mlflow experiment
        tracking_uri (string): mlflow tracking uri
        artifact_location (string): path of mlflow artifacts
    Returns:
        mlflow client & expt_id
    """
    client = mlflow.tracking.MlflowClient(tracking_uri=tracking_uri)
    try:
        expt = client.get_experiment_by_name(expt_name)
        expt_id = expt.experiment_id
    except Exception as e:
        type(e)
        expt_id = client.create_experiment(
            expt_name, artifact_location=artifact_location
        )

    mlflow.set_tracking_uri(tracking_uri)
    mlflow.set_experiment(expt_name)

    return(client, expt_id)

In [0]:
import ta_lib

In [0]:
# AutoML
import numpy as np
import pandas as pd
from tigerml.automl import *

# Reading data
c_data = spark.sql('select * from demo.titanic_raw').toPandas()
c_data['Cabin'] = c_data['Cabin'].fillna(value=np.nan)
c_data['Embarked'] = c_data['Embarked'].fillna(value=np.nan)

# AutoML Config
config = {
  "table_name": "demo.titanic_raw",
  "model_type": "Classification",
  "generations": 30,
  "population_size": 50,
  "name": "titanic",
  "task": TASKS.classification,
  "drop_cols": ['PassengerId', 'Name', 'Ticket'],
  "target": "Survived",
  "train_size": 0.75,
  "remove_na": False
}

# AutoML Run
classifier = AutoML(
 generations=config["generations"],
 population_size=config["population_size"],
 name=config["name"],
 task=config["task"], #use TASKS.regression for regression.
)

# type(classifier)
classifier.prep_data(
 data=c_data.drop(config["drop_cols"], axis=1),
 dv_name=config["target"],
 train_size=config["train_size"],
 remove_na=config["remove_na"]
)
classifier.fit()

# Download this file to local by running following command in local machine, make sure databricks CLI is configured using token
# databricks fs cp dbfs:/FileStore/code-templates/output_reports/<file_name> <target_path_in_local_without_filename/.>
# Example Filename - dbfs:/FileStore/code-templates/output_reports/titanic_structured_classification_at_2021-04-28_15-31-19.html
reports_path = "/dbfs/FileStore/code-templates/output_reports/"
mertrics = classifier.get_report(no_of_pipelines=5, format='.html', reports_path = reports_path)

/databricks/python/lib/python3.7/site-packages/tpot/base.py:31: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
 import imp
/databricks/python/lib/python3.7/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
 warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")
/databricks/spark/python/pyspark/sql/pandas/conversion.py:92: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
 PyArrow >= 1.0.0 must be installed; however, your version was 0.15.1.
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
 warnings.warn(msg)
Version 0.11.5 of tpot is outdated. Version 0.11.7 was released Wednesday January 06, 2021.
Optimization Progress: 0%| | 0/1550 [00:00<?, ?pipeline/s]Optimization Progress: 0%| | 1/1550 [00:07<3:12:53, 7.47s/pipeline]Optimization Progress: 1%| | 9/1550 [00:10<2:16:57, 5.33s/pipeline]Optimization Progress: 1%| | 17/1550 [00:12<1:37:23, 3.81s/pipeline]Optimization Progress: 2%|▏ | 25/1550 [00:13<1:09:06, 2.72s/pipeline]Optimization Progress: 2%|▏ | 33/1550 [00:16<51:02, 2.02s/pipeline] Optimization Progress: 3%|▎ | 41/1550 [00:18<36:48, 1.46s/pipeline]Optimization Progress: 3%|▎ | 49/1550 [00:18<26:27, 1.06s/pipeline]Optimization Progress: 3%|▎ | 50/1550 [00:19<20:58, 1.19pipeline/s]Optimization Progress: 3%|▎ | 52/1550 [00:24<35:13, 1.41s/pipeline]Optimization Progress: 3%|▎ | 53/1550 [00:28<53:39, 2.15s/pipeline]Optimization Progress: 4%|▍ | 61/1550 [00:30<38:50, 1.57s/pipeline]Optimization Progress: 4%|▍ | 69/1550 [00:32<28:41, 1.16s/pipeline]Optimization Progress: 5%|▍ | 77/1550 [00:36<23:48, 1.03pipeline/s]Optimization Progress: 5%|▌ | 85/1550 [00:39<19:39, 1.24pipeline/s]Optimization Progress: 6%|▌ | 93/1550 [00:42<16:44, 1.45pipeline/s]
Generation 1 - Current best internal CV score: 0.752601110009221Optimization Progress: 7%|▋ | 101/1550 [00:44<13:11, 1.83pipeline/s]Optimization Progress: 7%|▋ | 104/1550 [00:51<26:04, 1.08s/pipeline]Optimization Progress: 7%|▋ | 105/1550 [00:54<41:55, 1.74s/pipeline]Optimization Progress: 7%|▋ | 113/1550 [00:58<32:42, 1.37s/pipeline]Optimization Progress: 8%|▊ | 121/1550 [01:04<27:36, 1.16s/pipeline]Optimization Progress: 8%|▊ | 129/1550 [01:05<20:38, 1.15pipeline/s]Optimization Progress: 9%|▉ | 137/1550 [01:09<17:28, 1.35pipeline/s]Optimization Progress: 9%|▉ | 145/1550 [01:22<23:45, 1.01s/pipeline]
Generation 2 - Current best internal CV score: 0.7526907545626946Optimization Progress: 10%|▉ | 151/1550 [01:23<17:44, 1.31pipeline/s]Optimization Progress: 10%|▉ | 152/1550 [01:32<1:15:53, 3.26s/pipeline]Optimization Progress: 10%|█ | 160/1550 [01:34<54:37, 2.36s/pipeline] Optimization Progress: 11%|█ | 168/1550 [01:37<40:15, 1.75s/pipeline]Optimization Progress: 11%|█▏ | 176/1550 [01:41<31:33, 1.38s/pipeline]Optimization Progress: 12%|█▏ | 184/1550 [01:45<25:48, 1.13s/pipeline]Optimization Progress: 12%|█▏ | 192/1550 [01:48<20:28, 1.11pipeline/s]Optimization Progress: 13%|█▎ | 200/1550 [01:49<15:03, 1.49pipeline/s]
Generation 3 - Current best internal CV score: 0.7526907545626946Optimization Progress: 13%|█▎ | 201/1550 [01:52<29:13, 1.30s/pipeline]Optimization Progress: 13%|█▎ | 202/1550 [01:58<59:15, 2.64s/pipeline]Optimization Progress: 13%|█▎ | 203/1550 [02:06<1:34:07, 4.19s/pipeline]Optimization Progress: 14%|█▎ | 211/1550 [02:09<1:08:19, 3.06s/pipeline]Optimization Progress: 14%|█▍ | 219/1550 [02:20<56:57, 2.57s/pipeline] Optimization Progress: 15%|█▍ | 227/1550 [02:24<42:56, 1.95s/pipeline]Optimization Progress: 15%|█▌ | 235/1550 [02:29<33:18, 1.52s/pipeline]Optimization Progress: 16%|█▌ | 243/1550 [02:32<25:37, 1.18s/pipeline]Optimization Progress: 16%|█▌ | 251/1550 [0

In [0]:
# MLFlow experiment setup
expt_name = "/Users/vinay.kumar@tigeranalytics.com/Code-Templates/AutoML Classifcation"
artifact_location = "dbfs:/FileStore/code-templates/mlflow_artifacts/automl/classification"
client, expt_id = init_mlflow(expt_name, artifact_location)
report_file = get_latest_file(reports_path)

# Register parameters
for key, val in config.items():
  mlflow.log_param(key, val)

# Register Report as artifact
mlflow.log_artifact(report_file, "Report_path")

In [0]:
%sh
pip list

Package Version Location
----------------------------- ------------------- --------------------------------------
absl-py 0.10.0
alabaster 0.7.12
alembic 1.4.3
altair 4.1.0
appdirs 1.4.4
argon2-cffi 20.1.0
arviz 0.10.0
asn1crypto 1.4.0
astunparse 1.6.3
attrs 20.1.0
azure-core 1.8.2
azure-storage-blob 12.5.0
Babel 2.8.0
backcall 0.2.0
bandit 1.6.2
black 19.10b0
bleach 3.2.1
blinker 1.4
bokeh 2.2.3
brotlipy 0.7.0
cachetools 4.2.2
category-encoders 2.2.2
certifi 2020.6.20
cffi 1.14.0
cftime 1.2.1
chardet 3.0.4
click 7.1.2
cloudpickle 1.3.0
colorcet 2.0.2
configparser 5.0.1
cryptography 3.1.1
cycler 0.10.0
cytoolz 0.11.0
dask 2.30.0
databricks-cli 0.9.1
datashader 0.11.1
datashape 0.5.4
deap 1.3.1
decorator 4.4.2
defusedxml 0.6.0
distributed 2.30.0
docker 4.3.1
docker-pycreds 0.4.0
docutils 0.16
dparse 0.5.1
eli5 0.10.1
entrypoints 0.3
fastprogress 1.0.0
flake8 3.8.3
flake8-bandit 2.1.2
flake8-black 0.2.1
flake8-docstrings 1.5.0
flake8-isort 3.0.1
flake8-polyfill 1.0.2
Flask 1.1.2
fsspec 0.7.4
future 0.16.0
gast 0.3.2
gitdb 4.0.5
GitPython 3.1.3
google-auth 1.22.1
google-auth-oauthlib 0.4.1
google-pasta 0.2.0
gorilla 0.3.0
graphviz 0.14
great-expectations 0.13.4
grpcio 1.32.0
gunicorn 20.0.4
h5py 2.10.0
HeapDict 1.0.1
holoviews 1.13.4
hvplot 0.6.0
hypothesis 5.20.3
idna 2.10
imageio 2.9.0
imagesize 1.2.0
importlib-metadata 2.0.0
iniconfig 1.0.1
invoke 1.3.0
ipykernel 5.3.4
ipython 7.18.1
ipython-genutils 0.2.0
ipywidgets 7.5.1
isodate 0.6.0
isort 4.3.21
itsdangerous 1.1.0
jedi 0.17.2
Jinja2 2.11.2
joblib 0.17.0
json5 0.9.5
jsonpatch 1.32
jsonpointer 2.1
jsonschema 3.2.0
jupyter-client 6.1.7
jupyter-core 4.6.3
jupyter-sphinx 0.3.1
jupyterlab 2.2.6
jupyterlab-server 1.2.0
Keras-Preprocessing 1.1.2
kiwisolver 1.2.0
llvmlite 0.34.0
locket 0.2.0
luminol 0.4
lxml 4.6.1
Mako 1.1.3
Markdown 3.3.2
MarkupSafe 1.1.1
matplotlib 3.3.2
mccabe 0.6.1
mistune 0.8.4
mkl-fft 1.2.0
mkl-random 1.1.1
mkl-service 2.3.0
mlxtend 0.17.3
more-itertools 8.5.0
msgpack 1.0.0
msrest 0.6.19
multipledispatch 0.6.0
mypy-extensions 0.4.3
natsort 7.0.1
nbconvert 5.6.1
nbformat 5.0.7
nbsphinx 0.7.1
netCDF4 1.5.3
networkx 2.5
nose 1.3.7
notebook 6.1.4
numba 0.51.2
numpy 1.19.1
oauthlib 3.1.0
olefile 0.46
opt-einsum 3.3.0
packaging 20.4
pandas 1.0.5
pandas-flavor 0.2.0
pandocfilters 1.4.2
panel 0.9.7
param 1.9.3
parso 0.7.0
partd 1.1.0
pathspec 0.7.0
patsy 0.5.1
pbr 5.5.0
pexpect 4.8.0
pickleshare 0.7.5
Pillow 8.0.0
pip 20.2.2
pluggy 0.13.1
ply 3.11
pockets 0.9.1
prometheus-client 0.8.0
prometheus-flask-exporter 0.18.1
prompt-toolkit 3.0.8
protobuf 3.13.0
psutil 5.7.2
ptyprocess 0.6.0
py 1.9.0
pyarrow 0.15.1
pyasn1 0.4.8
pyasn1-modules 0.2.8
pycodestyle 2.6.0
pycparser 2.20
pyct 0.4.8
pydocstyle 5.1.1
pyflakes 2.2.0
Pygments 2.7.1
pygpu 0.7.6
pyjanitor 0.20.9
PyJWT 1.7.1
pymc3 3.9.3
Pyomo 5.7
pyOpenSSL 19.1.0
pyparsing 2.4.7
pyrsistent 0.17.3
PySocks 1.7.1
pytest 6.0.1
python-dateutil 2.8.1
python-editor 1.0.4
python-slugify 3.0.4
pytz 2020.1
PyUtilib 6.0.0
pyviz-comms 0.7.6
PyWavelets 1.1.1
PyYAML 5.3.1
pyzmq 19.0.2
querystring-parser 1.2.4
regex 2020.7.14
requests 2.23.0
requests-oauthlib 1.3.0
rsa 4.6
ruamel.yaml 0.17.2
ruamel.yaml.clib 0.2.2
ruptures 1.1.3
safety 1.9.0
scikit-image 0.17.2
scikit-learn 0.23.2
scipy 1.5.2
seaborn 0.10.1
Send2Trash 1.5.0
setuptools 50.3.0.post20201006
shap 0.35.0
singledispatch 3.4.0.3
six 1.15.0
smmap 3.0.4
snowballstemmer 2.0.0
sortedcontainers 2.2.2
Sphinx 3.2.1
sphinx-rtd-theme 0.5.0
sphinxcontrib-applehelp 1.0.2
sphinxcontrib-devhelp 1.0.2
sphinxcontrib-htmlhelp 1.0.3
sphinxcontrib-jsmath 1.0.1
sphinxcontrib-napoleon 0.7
sphinxcontrib-qthelp 1.0.3
sphinxcontrib-serializinghtml 1.1.4
SQLAlchemy 1.3.13
sqlparse 0.4.1
statsmodels 0.11.1
stevedore 3.2.2
stopit 1.1.2
ta-lib 1.0.0 /dbfs/FileStore/Git/code-templates/src
tabulate 0.8.7
tblib 1.7.0
tensorboard 2.2.0
tensorboard-plugin-wit 1.7.0
tensorflow 2.2.0
tensorflow-estimator 2.2.0
tensorflow-probability 0.10.1
termcolor 1.1.0
terminado 0.9.1
testfixtures 6.14.1
testpath 0.4.4
text-